<h1 style="text-align:center;">Performances des équipes au cours de la saison 25/26 / Team performance during the season 25/26 / Rendimiento de los equipos durante la temporada 25/26</h1>
 
**Subject**: Nous analyserons les performances d'une équipe au cours de la saison 25/26 en fonction de différents facteurs (buts, pressing, séquence de jeu, défense..). Ceci donnera lieu à un classement entre les équipes du Big 5. / We will analyse a team's performance during the 25/26 season based on various factors (goals, pressing, sequence of play, defence, etc.). This will result in a ranking of the Big 5 teams. / Analizaremos el rendimiento de un equipo durante la temporada 25/26 en función de diferentes factores (goles, presión, secuencia de juego, defensa, etc.). Esto dará lugar a una clasificación entre los equipos de la Big 5.

**Database**: Pour réaliser ce sujet, nous utiliserons deux sources : Opta et Fbref. / To carry out this subject, we will use two sources : Opta and Fbref. / Para abordar este tema, utilizaremos dos fuentes: Opta y Fbref.

## <span style="color:orange">**Task 1 :**</span> Extraction des données / Data extraction / Extracción de datos

In [1]:
# Importer les librairies / Importing libraries / Importar las bibliotecas
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
from scipy.stats import zscore
from scipy.stats import rankdata
from pathlib import Path
import unicodedata
import re

In [2]:
def _get_script_dir():
    try:
        return Path(__file__).resolve().parent
    except NameError:
        return Path.cwd()

# Localisation des fichiers / File location / Ubicación de los archivos
script_dir = _get_script_dir()
data_team_dir = script_dir.parent.parent / "data" / "team"

# Chemins des fichiers / path of this files / La ruta de acceso a este archivos
team_path = data_team_dir / "fbref_analyst_joined.csv"

# Récupération des données / Data recovery / Recuperación de datos
team_data = pd.read_csv(team_path)

In [3]:
# On affiche les 1ères lignes du fichier / The 1st lines of the file are displayed / Se muestran las primeras líneas del archivo.
team_data.head()

,id_season,team_id,season_name,country,championship_name,team_code,team_logo,attacking_overall__played,attacking_overall__goals,attacking_overall__xg,...,Clr__def,Err__def,Weekly Wages,Annual Wages,% Estimated,pts_league,rank_league,Long_Att__pass_prop,direct_attack_prop,build_ups_prop
0,1,144,2025/26,France,Ligue 1,Marseille,https://omo.akamai.opta.net/image.php?secure=t...,8,2.63,2.20,...,137,5,1835000.0,95420000.0,81%,18,1,10.22,24.60,75.40
1,1,149,2025/26,France,Ligue 1,Paris S-G,https://omo.akamai.opta.net/image.php?secure=t...,8,2.00,1.70,...,110,5,3361346.0,174790000.0,46%,17,2,8.49,20.69,79.31
2,1,153,2025/26,France,Ligue 1,Strasbourg,https://omo.akamai.opta.net/image.php?secure=t...,8,2.13,2.00,...,184,6,637885.0,33170000.0,100%,16,3,10.91,38.46,61.54
3,1,142,2025/26,France,Ligue 1,Lens,https://omo.akamai.opta.net/image.php?secure=t...,8,1.50,1.78,...,198,4,746923.0,38840000.0,100%,16,4,16.03,66.54,33.46
4,1,143,2025/26,France,Ligue 1,Lyon,https://omo.akamai.opta.net/image.php?secure=t...,8,1.38,1.89,...,184,6,994808.0,51730000.0,91%,15,5,12.74,29.59,70.41


## <span style="color:orange">**Task 2 :**</span> Rating / Notation

In [35]:
# Chargement du fichier / Load file / Cargando el archivo
df = pd.read_csv("../../data/team/fbref_analyst_joined.csv")

# Définir la liste de colonne / Define columns / Definir la lista de columnas
stat_cols = df.columns[8:]

# Inverser les statistiques où un chiffre élevé est une indication d'une sous-performance
# Reversing statistics where a high figure is an indication of underperformance
# Invertir las estadísticas en las que una cifra elevada es indicativa de un rendimiento inferior al esperado
inverted_stats = ['Err_per90', 'PKcon_per90', 'CrdR_per90', 'CrdY_per90', 'Fls_per90', 'Mis_per90', 'Dis_per90',
                  'GA_per90', 'SoTA_per90', 'PSxG/SoT', 'PKm_per90','PSxG_per_90','PSxG', 'Pkm_per90']

# Catégorie des postes / Position categories / Categoría de puestos
position_category = {
    "Goalkeeper": "Goalkeepers",
    "Centre-Back": "Central Defenders",
    "Right-Back": "Fullbacks",
    "Left-Back": "Fullbacks",
    "Left Midfield": "Midfielders",
    "Right Midfield": "Midfielders",
    "Central Midfield": "Midfielders",
    "Defensive Midfield": "Midfielders",
    "Attacking Midfield": "Attacking Midfielders / Wingers",
    "Right Winger": "Attacking Midfielders / Wingers",
    "Left Winger": "Attacking Midfielders / Wingers",
    "Second Striker": "Forwards",
    "Centre-Forward": "Forwards"
}
df["position_group"] = df["position"].map(position_category)

# Fonction de normalisation / Normalization function / Función de normalización
def min_max_normalize(series, inverse=False):
    min_val = series.min()
    max_val = series.max()
    if max_val == min_val:
        return pd.Series([0.5] * len(series), index=series.index)
    norm = (series - min_val) / (max_val - min_val)
    return 1 - norm if inverse else norm

# Application de la normalisation par groupe selon le poste / Apply normalization per position group
# Aplicación de la normalización por grupo según el puesto
normalized_df = (
    df.groupby("position_group")[stat_cols]
      .transform(lambda s: min_max_normalize(s, inverse=(s.name in inverted_stats)))
      .add_suffix("_norm")
)
df = pd.concat([df, normalized_df], axis=1, copy=False)

# Choix des statistiques et de leurs poids associés / Choice of statistics and their associated weights
# Selección de las estadísticas y sus ponderaciones asociadas
categories = {
    "goal_scoring_created": [(0.6, "npxG"),  (0.3, "npxG_per90"), (0.05, "SoT_per90"), (0.05, "Sh_per90")],
    "finish": [(0.5, "G-PK"), (0.2, "G-PK_per90"), (0.2, "G-xG"), (0.05, "G-xG_per90"),  (0.03, "G/Sh"), (0.02, "SoT%")],
    "building": [(0.4, "PrgP_per90"), (0.25, "Cmp_per90"), (0.15, "PrgR_per90"), (0.15, "Cmp%"), (0.025, "Sw_per90"), (0.025, "Crs_per90")],
    "creation": [(0.8, "xAG_per90"), (0.12, "Ast_per90"), (0.02, "1/3_per90"), (0.02, "PPA_per90"),
                 (0.02, "CrsPA_per90"), (0.02, "A-xAG")],
    "dribble": [(0.7, "Succ_per90"), (0.3, "Succ%")],
    "projection": [(0.6, "PrgC_per90"), (0.4, "Carries_per90")],
    "provoked_fouls": [(0.8, "Fld_per90"), (0.2, "PKwon_per90")],
    "waste": [(0.7, "Err_per90"), (0.15, "Mis_per90"), (0.15, "Dis_per90")],
    "defensive_actions": [(0.3, "Tkl%"), (0.3, "Int_per90"), (0.3, "Tkl_per90"), (0.05, "Recov_per90"), (0.05, "Clr_per90")],
    "faults_committed": [(0.4, "CrdY_per90"), (0.3, "CrdR_per90"),  (0.2, "Fls_per90"), (0.1, "PKcon_per90")],
    "aerial": [(0.7, "Won_per90"), (0.3, "Won%")]
}
goalkeeper_categories = {
    "goal_scoring_conceded": [(0.45, "GA_per90"), (0.25, "PSxG_per90"), (0.15, "PSxG"), (0.05, "SoTA_per90"), (0.05, "PSxG/SoT"), (0.05, "PKm_per90")],
    "efficiency": [(0.45, "/90"), (0.25, "Save%"), (0.15, "PSxG+/-"), (0.05, "Saves_per90"), (0.05, "PKsv_per90"), (0.05, "CS%")],
    "error_fouls": [(0.6, "Err_per90"), (0.2, "PKcon_per90"), (0.1, "CrdR_per90"), (0.05, "CrdY_per90"), (0.05, "Fls_per90")],
    "short_clearance": [(1.0, "Launch%")],
    "long_clearance": [(0.5, "AvgLen"), (0.3, "Cmp%"), (0.1, "PrgP_per90"), (0.1, "xAG_per90")],
    "positioning": [(0.7, "AvgDist"), (0.3, "#OPA_per90")],
    "aerial_defense": [(0.6, "Stp%"), (0.2, "Won%"), (0.1, "Stp_per90"), (0.1, "Won_per90")]

}

#  Calcul des scores par catégorie / Compute category scores / Cálculo de puntuaciones por categoría
def compute_category_score(row, stat_list):
    return 100 * sum(coef * row.get(f"{stat}_norm", 0) for coef, stat in stat_list)

for cat_name, stat_list in {**categories, **goalkeeper_categories}.items():
    df[f"score_{cat_name}_raw"] = df.apply(lambda row: compute_category_score(row, stat_list), axis=1)

# Normaliser les scores des catégories par groupe de postes en utilisant les percentiles (0-100)
# Normalize category scores per position_group using percentiles (0-100)
# Normalizar las puntuaciones de las categorías por grupo de puestos utilizando percentiles (0-100)
def percentile_rank(series):
    return 100 * (rankdata(series, method="min") - 1) / (len(series) - 1) if len(series) > 1 else pd.Series([50.0] * len(series), index=series.index)

for cat_name in categories.keys() | goalkeeper_categories.keys():
    raw_col = f"score_{cat_name}_raw"
    norm_col = f"score_{cat_name}"
    df[norm_col] = df.groupby("position_group")[raw_col].transform(percentile_rank).round(0).astype("Int64")
    df.drop(columns=raw_col, inplace=True)


# Poids associé aux catégories de statistique selon le poste du joueur / Weight associated with statistical categories by player position
# Peso asociado a las categorías estadísticas según la posición del jugador
position_weights = {
    "Centre-Back": {
        "goal_scoring_created": 0.06, "finish": 0.03, "building": 0.13, "creation": 0.06,
        "dribble": 0.02, "projection": 0.12, "provoked_fouls": 0.03, "waste": 0.05,
        "defensive_actions": 0.2, "faults_committed": 0.2, "aerial": 0.1
    },
    "Right-Back": {
        "goal_scoring_created": 0.14, "finish": 0.07, "building": 0.12, "creation": 0.09,
        "dribble": 0.02, "projection": 0.10, "provoked_fouls": 0.04, "waste": 0.05,
        "defensive_actions": 0.17, "faults_committed": 0.17, "aerial": 0.03
    },
    "Left-Back": {
        "goal_scoring_created": 0.14, "finish": 0.07, "building": 0.12, "creation": 0.09,
        "dribble": 0.02, "projection": 0.10, "provoked_fouls": 0.04, "waste": 0.05,
        "defensive_actions": 0.17, "faults_committed": 0.17, "aerial": 0.03
    },
    "Right Midfield": {
        "goal_scoring_created": 0.16, "finish": 0.08, "building": 0.12, "creation": 0.10,
        "dribble": 0.02, "projection": 0.12, "provoked_fouls": 0.05, "waste": 0.05,
        "defensive_actions": 0.125, "faults_committed": 0.125, "aerial": 0.05
    },
    "Left Midfield": {
        "goal_scoring_created": 0.16, "finish": 0.08, "building": 0.12, "creation": 0.10,
        "dribble": 0.02, "projection": 0.12, "provoked_fouls": 0.05, "waste": 0.05,
        "defensive_actions": 0.125, "faults_committed": 0.125, "aerial": 0.05
    },
    "Defensive Midfield": {
        "goal_scoring_created": 0.1, "finish": 0.05, "building": 0.14, "creation": 0.09,
        "dribble": 0.02, "projection": 0.13, "provoked_fouls": 0.05, "waste": 0.05,
        "defensive_actions": 0.16, "faults_committed": 0.16, "aerial": 0.06
    },
    "Central Midfield": {
        "goal_scoring_created": 0.2, "finish": 0.1, "building": 0.10, "creation": 0.10,
        "dribble": 0.02, "projection": 0.10, "provoked_fouls": 0.04, "waste": 0.04,
        "defensive_actions": 0.125, "faults_committed": 0.125, "aerial": 0.05
    },
    "Attacking Midfield": {
        "goal_scoring_created": 0.35, "finish": 0.15, "building": 0.08, "creation": 0.12,
        "dribble": 0.04, "projection": 0.08, "provoked_fouls": 0.06, "waste": 0.05,
        "defensive_actions": 0.03, "faults_committed": 0.03, "aerial": 0.01
    },
    "Right Winger": {
        "goal_scoring_created": 0.35, "finish": 0.15, "building": 0.06, "creation": 0.10,
        "dribble": 0.10, "projection": 0.06, "provoked_fouls": 0.06, "waste": 0.05,
        "defensive_actions": 0.03, "faults_committed": 0.03, "aerial": 0.01
    },
    "Left Winger": {
        "goal_scoring_created": 0.35, "finish": 0.15, "building": 0.06, "creation": 0.10,
        "dribble": 0.10, "projection": 0.06, "provoked_fouls": 0.06, "waste": 0.05,
        "defensive_actions": 0.03, "faults_committed": 0.03, "aerial": 0.01
    },
    "Second Striker": {
        "goal_scoring_created": 0.4, "finish": 0.15, "building": 0.08, "creation": 0.13,
        "dribble": 0.03, "projection": 0.08, "provoked_fouls": 0.04, "waste": 0.03,
        "defensive_actions": 0.025, "faults_committed": 0.025, "aerial": 0.01
    },
    "Centre-Forward": {
        "goal_scoring_created": 0.5, "finish": 0.2, "building": 0.03, "creation": 0.08,
        "dribble": 0.03, "projection": 0.03, "provoked_fouls": 0.04, "waste": 0.03,
        "defensive_actions": 0.025, "faults_committed": 0.025, "aerial": 0.01
    }
}

goalkeeper_weights = {
    "goal_scoring_conceded": 0.01, "efficiency": 0.74, "error_fouls": 0.18, "short_clearance": 0.01,
    "long_clearance": 0.01, "positioning": 0.01, "aerial_defense": 0.02
}

# Calcul de la note finale / Compute final rating / Cálculo de la nota final
def compute_rating(row):
    if row["position"] == "Goalkeeper":
        return sum(row[f"score_{cat}"] * weight for cat, weight in goalkeeper_weights.items())
    weights = position_weights.get(row["position"])
    if not weights:
        return None
    return sum(row[f"score_{cat}"] * weight for cat, weight in weights.items())

df["rating"] = df.apply(compute_rating, axis=1)

df["rating_raw"] = df["rating"]

# Équilibrer la note selon la position du joueur / Balance the rating according to the player's position
# Equilibrar la nota según la posición del intérprete
df["rating_raw"] = pd.to_numeric(df["rating"], errors="coerce")
df["rating_percentile"] = df.groupby("position_group")["rating_raw"].transform(percentile_rank)
df["rating"] = (0.4 * df["rating_raw"] + 0.6 * df["rating_percentile"]).round(0).astype("Int64")


# Arrondir les scores et la note / Round scores and rating / Redondear las puntuaciones y la nota
score_cols = [col for col in df.columns if col.startswith("score_")]
df[score_cols + ["rating"]] = df[score_cols + ["rating"]].round(0).astype("Int64")

# Power Ranking par championnat (source Opta Analyst) / Power Ranking by league (according to Opta Analyst) / Clasificación por campeonato (fuente: Opta Analyst)
power_ranking = {
    "GB1": 92.6,
    "IT1": 87,
    "ES1": 87,
    "L1": 86.3,
    "FR1": 85.3
}

# Référence = Power Ranking de GB1 / Benchmark = Power Ranking de GB1 / Referencia = Clasificación de poder de GB1
reference_ranking = power_ranking["GB1"]

# Appliquer une pénalité relative : ratio entre ranking / référence (max 1) / Apply a relative penalty: ranking/reference ratio (max 1)
# Aplicar una penalización relativa: relación entre clasificación y referencia (máximo 1).
df["power_ranking_raw"] = df["Comp"].map(power_ranking).fillna(85)
#df["power_ranking_penalty"] = df["power_ranking_raw"] / reference_ranking
df["power_ranking_penalty"] = 1 - (1 - (df["power_ranking_raw"] / reference_ranking)) / 3

# Maximum de minutes selon la ligue / Maximum minutes depending on the league / Máximo de minutos según la liga
comp_to_max_minutes = comp_max_minutes.astype(float).to_dict()

# Calcule du % de minutes jouées / Calculation of the % of minutes played / Cálculo del porcentaje de minutos jugados
df = df.copy()
df["Min"] = pd.to_numeric(df["Min"], errors="coerce")

# Calcule du nombre de minutes jouées par championnat / Calculate the number of minutes played per championship / Cálculo del número de minutos jugados por campeonato
df["max_minute_season"] = df["Comp"].map(comp_to_max_minutes)

# Calcule du ratio du minutes jouées par joueur / Calculation of the ratio of minutes played per player / Cálculo del ratio de minutos jugados por jugador
df["minute_ratio"] = df["Min"] / df["max_minute_season"]


# Pénalité logistique graduelle : pas de pénalité si ratio ≥ 0.6, max 0.8 si ratio ≤ 0.15
# Graduated logistical penalty: no penalty if ratio ≥ 0.6, max 0.8 if ratio ≤ 0.15
# Penalización logística gradual: sin penalización si la ratio es ≥ 0,6, máximo 0,8 si la ratio es ≤ 0,15
def compute_minutes_penalty(ratio):
    if ratio >= 0.6:
        return 1.0
    elif ratio <= 0.33:
        return 0.85
    else:
        return 0.85 + 0.15 * (ratio - 0.33) / (0.6 - 0.33)

df["minutes_penalty"] = df["minute_ratio"].apply(compute_minutes_penalty)

df["rating"] = (df["rating"] * df["power_ranking_penalty"] * df["minutes_penalty"]).round(0).astype("Int64")


# Sauvegarde du dataframe final / Save final DataFrame / Guardar el marco de datos final
df = df[[col for col in df.columns if not col.endswith('_norm') and col != "position_group"]]
df.drop(columns=["power_ranking_raw", "power_ranking_penalty", "max_minute_season", "minute_ratio",
                 "minutes_penalty", "rating_raw", "rating_percentile"  ], inplace=True)

# Liste des colonnes dans l’ordre désiré / List of column in the desired order / Lista de columnas en el orden deseado
ordered_score_cols = [
    "player_name", "player_id", "nationality", "Age", "Born", "position", "position_other", "height","foot","shirtNumber",
    "joinedOn", "contract", "Comp","club_name", "marketValue", "imageUrl","agent_name", "outfitter","status","rating",
    "score_goal_scoring_created","score_finish", "score_building", "score_creation","score_dribble","score_projection",
    "score_defensive_actions", "score_waste", "score_faults_committed", "score_provoked_fouls","score_aerial",
    "score_goal_scoring_conceded", "score_efficiency", "score_error_fouls","score_short_clearance",
    "score_long_clearance","score_positioning", "score_aerial_defense"
]

other_cols = [col for col in df.columns if col not in ordered_score_cols and not col.endswith('_norm') and col != "position_group"]
df = df[ordered_score_cols + other_cols]

df.to_csv("../../data/team/database_team.csv", index=False)